In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
# Create GIF
def create_gif(image_list, gif_name, ds):
    frames = []
    for image_name in image_list:
        if image_name.endswith('.png'):
            print(image_name)
            frames.append(imageio.imread(image_name))
    # Save them as frames into a gif
    imageio.mimsave(gif_name, frames, 'GIF', duration = ds)
    return
import openpyxl

In [ ]:
# This code aims scan and save runable combination of parameter values

In [ ]:
# Define a function to calculate concentration change, whether electrolyte being squeezed out or added in
def Cal_new_con_Update(Sol,Para):   # subscript r means the reservoir
    # Note: c_EC_r is the initial EC  concentraiton in the reservoir; 
    #       c_e_r  is the initial Li+ concentraiton in the reservoir;
    #############################################################################################################################  
    ###############################           Step-1 Prepare parameters:        #################################################
    #############################################################################################################################  
    L_p   =Para["Positive electrode thickness [m]"]
    L_n   =Para["Negative electrode thickness [m]"]
    L_s   =Para["Separator thickness [m]"]
    L_y   =Para["Electrode width [m]"]   # Also update to change A_cc and therefore Q
    L_z   =Para["Electrode height [m]"]
    c_EC_r_old=Para["Current solvent concentration in the reservoir [mol.m-3]"]   # add two new parameters for paper 2
    c_e_r_old =Para["Current electrolyte concentration in the reservoir [mol.m-3]"]
    # Initial EC concentration in JR
    c_EC_JR_old =Para["Bulk solvent concentration [mol.m-3]"]  # Be careful when multiplying with pore volume to get amount in mole. Because of electrolyte dry out, that will give more amount than real.   
    # LLI due to electrode,Ratio of EC and lithium is 1:1 -> EC amount consumed is LLINegSEI[-1]
    LLINegSEI = Sol["Loss of lithium to negative electrode SEI [mol]"].entries[-1] - Sol["Loss of lithium to negative electrode SEI [mol]"].entries[0]
    LLINegDeadLiP = Sol["Loss of lithium to negative electrode dead lithium plating [mol]"].entries[-1] - Sol["Loss of lithium to negative electrode dead lithium plating [mol]"].entries[0]
    LLINegLiP = Sol["Loss of lithium to negative electrode lithium plating [mol]"].entries[-1] - Sol["Loss of lithium to negative electrode lithium plating [mol]"].entries[0]
    cLi_Xavg  = Sol["X-averaged electrolyte concentration [mol.m-3]"].entries[-1] 
    # Pore volume change with time:
    PoreVolNeg_0 = Sol["X-averaged negative electrode porosity"].entries[0]*L_n*L_y*L_z;
    PoreVolSep_0 = Sol["X-averaged separator porosity"].entries[0]*L_s*L_y*L_z;
    PoreVolPos_0 = Sol["X-averaged positive electrode porosity"].entries[0]*L_p*L_y*L_z;
    PoreVolNeg_1 = Sol["X-averaged negative electrode porosity"].entries[-1]*L_n*L_y*L_z;
    PoreVolSep_1 = Sol["X-averaged separator porosity"].entries[-1]*L_s*L_y*L_z;
    PoreVolPos_1 = Sol["X-averaged positive electrode porosity"].entries[-1]*L_p*L_y*L_z;
    #############################################################################################################################  
    ##### Step-2 Determine How much electrolyte is added (from the reservoir to JR) or squeezed out (from JR to reservoir) ######
    #######################   and finish electrolyte mixing     #################################################################  
    Vol_Elely_Tot_old = Para["Current total electrolyte volume in whole cell [m3]"] 
    Vol_Elely_JR_old  = Para["Current total electrolyte volume in jelly roll [m3]"] 
    if Vol_Elely_Tot_old - Vol_Elely_JR_old < 0:
        print('Model error! Electrolyte in JR is larger than in the cell!')
    Vol_Pore_tot_old  = PoreVolNeg_0 + PoreVolSep_0 + PoreVolPos_0    # pore volume at start time of the run
    Vol_Pore_tot_new  = PoreVolNeg_1 + PoreVolSep_1 + PoreVolPos_1    # pore volume at end   time of the run, intrinsic variable 
    Vol_Pore_decrease = Vol_Elely_JR_old  - Vol_Pore_tot_new;
    # EC:lithium:SEI=2:2:1     for SEI=(CH2OCO2Li)2
    # Because inner and outer SEI partial molar volume is the same, just set one for whole SEI
    VmolSEI   = Para["Outer SEI partial molar volume [m3.mol-1]"] # 9.8e-5,
    VmolLiP   = Para["Lithium metal partial molar volume [m3.mol-1]"] # 1.3e-05
    VmolEC    = 6.667e-5  # Unit:m3.mol-1; According to Wiki, correct value: EC molar volume is :66.67 cm3.mol-1  = 6.667e-5, 
    Vol_EC_consumed  =  ( LLINegSEI + LLINegDeadLiP  ) * VmolEC    # Mark: Ruihe add LLINegDeadLiP, either with 2 or not, will decide how fast electrolyte being consumed!
    Vol_Elely_need   = Vol_EC_consumed - Vol_Pore_decrease
    Vol_SEILiP_increase = 0.5*(LLINegSEI * VmolSEI + LLINegLiP * VmolLiP)    #  volume increase due to SEI+total LiP 
    Test_V = Vol_SEILiP_increase - Vol_Pore_decrease  #  This value should always be zero, but now not, which becomes the source of error!
    Test_V2= (Vol_Pore_tot_old - Vol_Elely_JR_old) / Vol_Elely_JR_old * 100; # Porosity errors due to first time step
    
    # Start from here, there are serveral variables to be determined:
    #   1) Vol_Elely_add, or Vol_Elely_squeezed; depends on conditions. easier for 'squeezed' condition
    #   2) Vol_Elely_Tot_new, should always equals to Vol_Elely_Tot_old -  Vol_EC_consumed;
    #   3) Vol_Elely_JR_new, for 'add' condition: see old code; for 'squeezed' condition, equals to pore volume in JR
    #   4) Ratio_Dryout, for 'add' condition: see old code; for 'squeezed' condition, equals to Vol_Elely_Tot_new/Vol_Pore_tot_new 
    #   5) Ratio_CeEC_JR and Ratio_CeLi_JR: for 'add' condition: see old code; for 'squeezed' condition, equals to 1 (unchanged)
    #   6) c_e_r_new and c_EC_r_new: for 'add' condition: equals to old ones (unchanged); for 'squeezed' condition, need to carefully calculate     
    #   7) Width_new: for 'add' condition: see old code; for 'squeezed' condition, equals to L_y (unchanged)
    Vol_Elely_Tot_new = Vol_Elely_Tot_old - Vol_EC_consumed;
    
    if Vol_Elely_need < 0:
        print('Electrolyte is being squeezed out, check plated lithium (reversible part)')
        Vol_Elely_squeezed = - Vol_Elely_need;   # Make Vol_Elely_squeezed>0 for simplicity 
        Vol_Elely_add = 0.0;
        Vol_Elely_JR_new = Vol_Pore_tot_new; 
        Ratio_Dryout = Vol_Elely_Tot_new / Vol_Elely_JR_new
        Ratio_CeEC_JR = 1.0;    # the concentrations in JR don't need to change
        Ratio_CeLi_JR = 1.0; 
        SqueezedLiMol   =  Vol_Elely_squeezed*cLi_Xavg;
        SqueezedECMol   =  Vol_Elely_squeezed*c_EC_JR_old;
        Vol_Elely_reservoir_old = Vol_Elely_Tot_old - Vol_Elely_JR_old; 
        LiMol_reservoir_new = Vol_Elely_reservoir_old*c_e_r_old  + SqueezedLiMol;
        ECMol_reservoir_new = Vol_Elely_reservoir_old*c_EC_r_old + SqueezedECMol;
        c_e_r_new= LiMol_reservoir_new / (Vol_Elely_reservoir_old+Vol_Elely_squeezed);
        c_EC_r_new= ECMol_reservoir_new / (Vol_Elely_reservoir_old+Vol_Elely_squeezed);
        Width_new = L_y; 
    else:   # this means Vol_Elely_need >= 0, therefore the folliwing script should works for Vol_Elely_need=0 as well!
        # Important: Calculate added electrolyte based on excessive electrolyte, can be: 1) added as required; 2) added some, but less than required; 3) added 0 
        Vol_Elely_squeezed = 0;  
        if Vol_Elely_Tot_old > Vol_Elely_JR_old:                             # This means Vol_Elely_JR_old = Vol_Pore_tot_old ()
            if Vol_Elely_Tot_old-Vol_Elely_JR_old >= Vol_Elely_need:         # 1) added as required
                Vol_Elely_add     = Vol_Elely_need;  
                Vol_Elely_JR_new  = Vol_Pore_tot_new;  # also equals to 'Vol_Pore_tot_new', or Vol_Pore_tot_old - Vol_Pore_decrease
                Ratio_Dryout = 1.0;
            else:                                                            # 2) added some, but less than required;                                                         
                Vol_Elely_add     = Vol_Elely_Tot_old - Vol_Elely_JR_old;   
                Vol_Elely_JR_new  = Vol_Elely_Tot_new;                       # This means Vol_Elely_JR_new <= Vol_Pore_tot_new
                Ratio_Dryout = Vol_Elely_JR_new/Vol_Pore_tot_new;
        else:                                                                # 3) added 0 
            Vol_Elely_add = 0;
            Vol_Elely_JR_new  = Vol_Elely_Tot_new; 
            Ratio_Dryout = Vol_Elely_JR_new/Vol_Pore_tot_new;

        # Next: start mix electrolyte based on previous defined equation
        # Lithium amount in liquid phase, at initial time point
        TotLi_Elely_JR_Old = Sol["Total lithium in electrolyte [mol]"].entries[-1]; # remember this is only in JR
        # Added lithium and EC amount in the added lithium electrolyte: - 
        AddLiMol   =  Vol_Elely_add*c_e_r_old
        AddECMol   =  Vol_Elely_add*c_EC_r_old
        # Total amount of Li and EC in current electrolyte:
        # Remember Li has two parts, initial and added; EC has three parts, initial, consumed and added
        TotLi_Elely_JR_New   = TotLi_Elely_JR_Old + AddLiMol
        TotECMol_JR   = Vol_Elely_JR_old*c_EC_JR_old - LLINegSEI + AddECMol  # EC:lithium:SEI=2:2:1     for SEI=(CH2OCO2Li)2
        Ratio_CeEC_JR  = TotECMol_JR    /   Vol_Elely_JR_new   / c_EC_JR_old
        Ratio_CeLi_JR  = TotLi_Elely_JR_New    /   TotLi_Elely_JR_Old   /  Ratio_Dryout # Mark, change on 21-11-19
        c_e_r_new  = c_e_r_old;
        c_EC_r_new = c_EC_r_old;
        Width_new   = Ratio_Dryout * L_y;
        
    # Collect the above parameter in Data_Pack to shorten the code  
    Data_Pack   = [
        Vol_EC_consumed, 
        Vol_Elely_need, 
        Test_V, 
        Test_V2, 
        Vol_Elely_add, 
        Vol_Elely_Tot_new, 
        Vol_Elely_JR_new, 
        Vol_Pore_tot_new, 
        Vol_Pore_decrease, 
        c_e_r_new, c_EC_r_new,
        Ratio_Dryout, Ratio_CeEC_JR, 
        Ratio_CeLi_JR,
        Width_new, 
        ]  # 16 in total
    #print('Loss of lithium to negative electrode SEI', LLINegSEI, 'mol') 
    #print('Loss of lithium to negative electrode dead lithium plating', LLINegDeadLiP, 'mol') 
    #############################################################################################################################  
    ###################       Step-4 Update parameters here        ##############################################################
    #############################################################################################################################
    Para.update(   {'Bulk solvent concentration [mol.m-3]':  c_EC_JR_old * Ratio_CeEC_JR  })
    Para.update(   {'Ratio of Li-ion concentration change in electrolyte consider solvent consumption':  
                    Ratio_CeLi_JR })
    Para.update(   {'Current total electrolyte volume in whole cell [m3]':  Vol_Elely_Tot_new  })
    Para.update(   {'Current total electrolyte volume in jelly roll [m3]':  Vol_Elely_JR_new  })
    Para.update(   {'Ratio of electrolyte dry out in jelly roll':Ratio_Dryout})
    Para.update(   {'Electrode width [m]':Width_new})    
    Para.update(   {'Current solvent concentration in the reservoir [mol.m-3]':c_EC_r_new})     
    Para.update(   {'Current electrolyte concentration in the reservoir [mol.m-3]':c_e_r_new})             
    return Data_Pack,Para

In [ ]:
# Define a function to calculate based on previous solution
def Run_Model_Base_On_Last_Solution( Model  , Sol , Para_update, ModelExperiment, Update_Cycles ):
    # Use Sulzer's method: inplace = false
    ModelTimer = pb.Timer()
    # Important line: define new model based on previous solution
    Ratio_CeLi = Para_update["Ratio of Li-ion concentration change in electrolyte consider solvent consumption"]
    dict_short = {}; 
    list_short = [  "Discharge capacity [A.h]",   # mark Ruihe Li change for FullyCoupled as this is an old version of PyBaMM
        "Negative particle concentration",
        "Positive particle concentration",
        "Negative electrode potential",
        "Positive electrode potential",
        "Total negative electrode interfacial current density variable" ,
        "Total positive electrode interfacial current density variable",
        "Negative electrode porosity",
        "Separator porosity",
        "Positive electrode porosity",
        "Inner negative electrode SEI thickness",
        "Outer negative electrode SEI thickness",
        "Negative electrolyte potential",
        "Separator electrolyte potential",
        "Positive electrolyte potential",
        "Negative electrolyte concentration",
        "Separator electrolyte concentration",
        "Positive electrolyte concentration",   
        "Negative electrode lithium plating concentration",
        "Negative electrode dead lithium concentration",]
    for list_short_i in list_short:
        dict_short.update( { list_short_i : Sol.last_state[list_short_i].data  }  )
    dict_short["Negative electrolyte concentration"] = dict_short["Negative electrolyte concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Separator electrolyte concentration"] = dict_short["Separator electrolyte concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Positive electrolyte concentration"] = dict_short["Positive electrolyte concentration"] * Ratio_CeLi # important: update sol here!
    ModelModel_new = Model.set_initial_conditions_from(dict_short, inplace=False)
    #Para_update.update(   {'Ambient temperature [K]':313.15 });   # run model at 45 degree C
    #Para_update.update(   {'Initial temperature [K]':313.15 });
    ModelSimnew = pb.Simulation(
        ModelModel_new,
        experiment = ModelExperiment, 
        parameter_values=Para_update, 
        solver = pb.CasadiSolver(),
    )
    Sol_new = ModelSimnew.solve(save_at_cycles = Update_Cycles) # save every several cycles, can save RAM greatly
    # print("Solved this model in {}".format(ModelTimer.time()))
    return ModelModel_new, Sol_new
def Run_Model_Base_On_Last_Solution_RPT( Model  , Sol,  Para_update, ModelExperiment ,Update_Cycles):
    # Use Sulzer's method: inplace = false
    ModelTimer = pb.Timer()
    Ratio_CeLi = Para_update["Ratio of Li-ion concentration change in electrolyte consider solvent consumption"]
    # print("Model is now using average EC Concentration of:",Para_update['Bulk solvent concentration [mol.m-3]'])
    # print("Ratio of electrolyte dry out in jelly roll is:",Para_update['Ratio of electrolyte dry out in jelly roll'])
    # print("Model is now using an electrode width of:",Para_update['Electrode width [m]'])
    # Important line: define new model based on previous solution
    dict_short = {}; 
    list_short = [  "Discharge capacity [A.h]", # mark Ruihe Li change for FullyCoupled as this is an old version of PyBaMM
        "Negative particle concentration",
        "Positive particle concentration",
        "Negative electrode potential",
        "Positive electrode potential",
        "Total negative electrode interfacial current density variable" ,
        "Total positive electrode interfacial current density variable",
        "Negative electrode porosity",
        "Separator porosity",
        "Positive electrode porosity",
        "Inner negative electrode SEI thickness",
        "Outer negative electrode SEI thickness",
        "Negative electrolyte potential",
        "Separator electrolyte potential",
        "Positive electrolyte potential",
        "Negative electrolyte concentration",
        "Separator electrolyte concentration",
        "Positive electrolyte concentration",   
        "Negative electrode lithium plating concentration",
        "Negative electrode dead lithium concentration", ]
    for list_short_i in list_short:
        dict_short.update( { list_short_i : Sol.last_state[list_short_i].data  }  )
    dict_short["Negative electrolyte concentration"] = dict_short["Negative electrolyte concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Separator electrolyte concentration"]          = dict_short["Separator electrolyte concentration"] * Ratio_CeLi          # important: update sol here!
    dict_short["Positive electrolyte concentration"] = dict_short["Positive electrolyte concentration"] * Ratio_CeLi # important: update sol here!
    ModelModel_new = Model.set_initial_conditions_from(dict_short, inplace=False)
    Para_update.update(   {'Ambient temperature [K]':298.15 });
    Para_update.update(   {'Initial temperature [K]':298.15 });
    ModelSimnew = pb.Simulation(
        ModelModel_new,
        experiment = ModelExperiment, 
        parameter_values=Para_update, 
        solver = pb.CasadiSolver(),
    )
    Sol_new = ModelSimnew.solve(save_at_cycles = Update_Cycles) # save every several cycles, can save RAM greatly
    # print("Solved this model in {}".format(ModelTimer.time()))
    return ModelModel_new, Sol_new

In [ ]:
# Define electrolyte diffusivity and conductivity
def electrolyte_diffusivity_Nyman2008(c_e, T):
    if c_e < 2000:
        D_c_e = 8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10
    else:
        D_c_e = 8.794e-11 * (2000 / 1000) ** 2 - 3.972e-10 * (2000 / 1000) + 4.862e-10
    return D_c_e    
def electrolyte_conductivity_Nyman2008(c_e, T):
    if c_e < 2000:
        sigma_e = (0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000))
    else:
        sigma_e = (0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000))
    return sigma_e

In [ ]:
def write_excel_xlsx(path, sheet_name, value):
    index = len(value)
    workbook = openpyxl.Workbook()  # 新建工作簿（默认有一个sheet？）
    sheet = workbook.active  # 获得当前活跃的工作页，默认为第一个工作页
    sheet.title = sheet_name  # 给sheet页的title赋值
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.cell(row=i + 1, column=j + 1, value=str(value[i][j]))  # 行，列，值 这里是从1开始计数的
    workbook.save(path)  # 一定要保存
    print("xlsx格式表格写入数据成功！")
 

In [ ]:
# ################        Global setting  ################################# #################################
for i in range(0,1):  
    Total_Cycles = 1170; Cycle_bt_RPT = 78; Update_Cycles = 26;  Small_Loop =  int(Cycle_bt_RPT/Update_Cycles);   SaveTimes = int(Total_Cycles/Cycle_bt_RPT);   RPT_Cycles = 1; 

    # Special cases for Non-dry-out model:
    Total_Cycles_NonDry = 1170;   SaveTimes_NonDry = int(Total_Cycles_NonDry/Cycle_bt_RPT);  

    Real_SaveAs_Cycles = 1 # Set this to use it in real experiment so that can really get capacity throughput!
    V_max = 4.2;        V_min = 2.5;
    cs_Neg_Init = 28920; Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
    Experiment_Long   = pb.Experiment(  [(f"Discharge at 1 C until {V_min} V", f"Charge at 0.3 C until {V_max} V", f"Hold at {V_max} V until C/100"),  ] * Update_Cycles  )  
    Experiment_RPT    = pb.Experiment( [ (f"Discharge at 0.1C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1C until {V_max} V" ) ] * RPT_Cycles ) 
    Experiment_Breakin= pb.Experiment( [ ( f"Discharge at 0.1C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1C until {V_max} V" )  ] *2 )
    BasicPath = 'D:/OneDrive - Imperial College London/PhD Study/Imperial Study/Model/Pybamm Learning/SimDataSave/2nd paperTest R1/';
    Target  = 'Big scan office/'
    if not os.path.exists(BasicPath + Target):
        os.mkdir(BasicPath + Target);
    book_name_xlsx = 'Scan Chen2020_coupled 1170 cycles.xlsx'

    sheet_name_xlsx = 'Results'
    
    value3 = [
        ["Index", "cs_Neg_Init", "Diff_SEI", "R_SEI", 
        "Bulk_Sol_Con","D_Li_inSEI", "c_Li_inte_ref",
        "Couple_SEI_LiP","k_LiP","Cap Loss","LLI to LiP",
        "LLI to SEI","Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final"],
        ]
    write_excel_xlsx(BasicPath + Target+book_name_xlsx, sheet_name_xlsx, value3)
    ################ Important: index definition #################################
    index = list(np.arange(1,SaveTimes+1)*(Small_Loop+RPT_Cycles)-1); #index.insert(0,0)  # get all last ageing cycle before RPT, plus the first RPT test
    index2= list(np.arange(0,SaveTimes+1)*(Small_Loop+RPT_Cycles));         # get all RPT test
    cycles = np.arange(0,SaveTimes+1)*Cycle_bt_RPT; 
    cycles3 = list(np.arange(1,SaveTimes+1)*(Cycle_bt_RPT)); 

    index_NonDry    = list(np.arange(1,SaveTimes_NonDry+1)*(Small_Loop+RPT_Cycles)-1); #index.insert(0,0)  # get all last ageing cycle before RPT, plus the first RPT test
    index2_NonDry   = list(np.arange(0,SaveTimes_NonDry+1)*(Small_Loop+RPT_Cycles));         # get all RPT test  
    cycles_NonDry   = np.arange     (0,SaveTimes_NonDry+1)*Cycle_bt_RPT; 
    cycles3_NonDry  = list(np.arange(1,SaveTimes_NonDry+1)*(Cycle_bt_RPT)); 

In [4]:
ChemistryChen=pb.parameter_sets.Chen2020_coupled   
Para_0=pb.ParameterValues(chemistry=ChemistryChen)
Para_0.search("lithium")

Initial plated lithium concentration [mol.m-3]	0.0
Inner SEI lithium interstitial diffusivity [m2.s-1]	1e-20
Lithium interstitial reference concentration [mol.m-3]	15.0
Lithium metal partial molar volume [m3.mol-1]	1.3e-05
Lithium plating kinetic rate constant [m.s-1]	1e-10


In [ ]:
# Run model with electrolyte dry out
#Diff_OutSEI = [0.5000000000000002e-22,2.5000000000000002e-22,7.5000000000000002e-22]; Diff_OutSEI_Succeed = [];

D_Li_inSEI = [1e-20,1e-18, 1e-16, 1e-14];    # default: 1e-20 
c_Li_inte_ref = [10,15,20,25];    # default: 15
Couple_SEI_LiP = [1e-6,2E-6,1e-5,1e-4]; # default: 1e-6
k_LiP = [1E-10,1e-8,1e-6,];         # default: 1e-10
index = 0
for D_Li_inSEI_i in D_Li_inSEI:
    for c_Li_inte_ref_i in c_Li_inte_ref:
        for Couple_SEI_LiP_i in Couple_SEI_LiP:
            for k_LiP_i in k_LiP:
                index = index + 1; 
                try:             
                    for i in range(0,1):    ######################   update parameter and run first RPT
                        ChemistryChen=pb.parameter_sets.Chen2020_coupled   
                        Para_0=pb.ParameterValues(chemistry=ChemistryChen)
                        Para_0.update({'SEI resistivity [Ohm.m]':R_SEI}) ;
                        Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI});
                        Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con});
                        Para_0.update({'Initial concentration in negative electrode [mol.m-3]':cs_Neg_Init })
                        Para_0.update({'Initial electrolyte excessive amount ratio':1.0})   # 
                        Para_0.update({'Inner SEI lithium interstitial diffusivity [m2.s-1]':D_Li_inSEI_i})       
                        Para_0.update({'Lithium interstitial reference concentration [mol.m-3]':c_Li_inte_ref_i}) 
                        Para_0.update({"SEI-plating coupling constant [s-1]": Couple_SEI_LiP_i})     
                        Para_0.update({'Lithium plating kinetic rate constant [m.s-1]':k_LiP_i})                
                        Para_0.update({'Current solvent concentration in the reservoir [mol.m-3]':Bulk_Sol_Con})     
                        Para_0.update({'Current electrolyte concentration in the reservoir [mol.m-3]':1000.0})           
                        Para_0.update({'Ratio of Li-ion concentration change in electrolyte consider solvent consumption':  
                                        1.0 })
                        Model_0 = pb.lithium_ion.DFN(     
                        options={
                            "particle": "Fickian diffusion",          
                            "SEI":"interstitial-diffusion limited",          
                            "SEI film resistance":"distributed",          
                            "SEI porosity change":"true",         
                            "lithium plating":"partially reversible"     } ) #
                        ModelTimer = pb.Timer()
                        Sim_0    = pb.Simulation(
                            Model_0,        experiment = Experiment_Breakin,
                            parameter_values = Para_0,
                            solver = pb.CasadiSolver(),) #mode="safe"
                        Sol_0    = Sim_0.solve()

                    for i in range(0,1):    ######################   initialize for post-processing before running subsequent model   
                        Vol_Elely_Tot_All  = [];       Vol_Elely_JR_All =[];         Vol_Pore_tot_All =[];
                        Ratio_CeEC_All     = [];       Ratio_CeLi_All = [];          Ratio_Dryout_All =[];
                        Vol_EC_consumed_All= [];       Vol_Elely_need_All = [];      Width_all        =[];
                        Vol_Elely_add_All  = [];       Vol_Pore_decrease_All =[];
                        Para_All           = [];       Model_All = [];               Sol_All_i =[]  ;    
                        Test_V_All=[];    Test_V2_All=[]; c_e_r_new_All=[]; c_EC_r_new_All=[]; 
                        # for RPT:
                        Cap_RPT_Dry_All=[]; CapLoss_LiP_Dry_All=[]; 
                        CapLoss_SEI_Dry_All=[];         Cap_Pos_Dry_All=[];          Cap_Neg_Dry_All=[]; 

                
                        T_0                  =  Para_0['Initial temperature [K]']
                        Porosity_Neg_0       =  Para_0['Negative electrode porosity']  
                        Porosity_Pos_0       =  Para_0['Positive electrode porosity']  
                        Porosity_Sep_0       =  Para_0['Separator porosity']  
                        cs_Neg_Max           =  Para_0["Maximum concentration in negative electrode [mol.m-3]"];
                        L_p                  =  Para_0["Positive electrode thickness [m]"]
                        L_n                  =  Para_0["Negative electrode thickness [m]"]
                        L_s                  =  Para_0["Separator thickness [m]"]
                        L_y_0                =  Para_0["Initial Electrode width [m]"]
                        L_z_0                =  Para_0["Initial Electrode height [m]"]
                        Int_ElelyExces_Ratio =  Para_0["Initial electrolyte excessive amount ratio"] 
                        x_n = Sol_0.cycles[0].steps[0]["x_n [m]"].entries[:,-1]; x = Sol_0.cycles[0].steps[0]["x [m]"].entries[:,-1];
                        Vol_Elely_Tot        =  ( L_n*Porosity_Neg_0 +  L_p*Porosity_Pos_0  +  L_s*Porosity_Sep_0  )  * L_y_0 * L_z_0 * Int_ElelyExces_Ratio # Set initial electrolyte amount [L] 
                        Vol_Elely_JR         =  ( L_n*Porosity_Neg_0 +  L_p*Porosity_Pos_0  +  L_s*Porosity_Sep_0  )  * L_y_0 * L_z_0
                        Vol_Pore_tot         =  ( L_n*Porosity_Neg_0 +  L_p*Porosity_Pos_0  +  L_s*Porosity_Sep_0  )  * L_y_0 * L_z_0
                        Ratio_CeEC           =  1.0   
                        Ratio_CeLi           =  1.0   
                        Ratio_Dryout         =  1.0
                        Vol_EC_consumed      =  0
                        Vol_Elely_need       =  0
                        Vol_Elely_add        =  0
                        Vol_Pore_decrease    =  0
                        print('Initial electrolyte amount is ', Vol_Elely_Tot*1e6, 'mL') 
                        Para_0.update({'Current total electrolyte volume in jelly roll [m3]':Vol_Elely_JR})
                        Para_0.update({'Current total electrolyte volume in whole cell [m3]':Vol_Elely_Tot})   
                        Vol_Elely_Tot_All.append(Vol_Elely_Tot*1e6);            Vol_Elely_JR_All.append(Vol_Elely_JR*1e6);     Vol_Pore_tot_All.append(Vol_Pore_tot*1e6);           
                        Ratio_CeEC_All.append(Ratio_CeEC);                      Ratio_CeLi_All.append(Ratio_CeLi);             Ratio_Dryout_All.append(Ratio_Dryout);
                        Vol_EC_consumed_All.append(Vol_EC_consumed*1e6);        Vol_Elely_need_All.append(Vol_Elely_need*1e6);     Width_all.append(L_y_0);
                        Vol_Elely_add_All.append(Vol_Elely_add*1e6);            Vol_Pore_decrease_All.append(Vol_Pore_decrease*1e6);

                    for i in range(0,1):    #################### post process for the first RPT cycle    ############################
                        Cap_RPT_Dry_All.append    (Sol_0.cycles[-1].steps[0]["Discharge capacity [A.h]"].entries[-1] - Sol_0.cycles[-1].steps[0]["Discharge capacity [A.h]"].entries[0])
                        CapLoss_LiP_Dry_All.append(Sol_0.cycles[-1].steps[0]["Loss of capacity to negative electrode lithium plating [A.h]"].entries[-1])
                        CapLoss_SEI_Dry_All.append(Sol_0.cycles[-1].steps[0]["Loss of capacity to negative electrode SEI [A.h]"].entries[-1])
                        Cap_Pos_Dry_All.append    (Sol_0.cycles[-1].steps[0]["Positive electrode capacity [A.h]"].entries[-1])
                        Cap_Neg_Dry_All.append    (Sol_0.cycles[-1].steps[0]["Negative electrode capacity [A.h]"].entries[-1])      
                    x_n = Sol_0.cycles[0].steps[0]["x_n [m]"].entries[:,-1]; x = Sol_0.cycles[0].steps[0]["x [m]"].entries[:,-1];   # Get location variable, can be any single solution

                    # Para_All.append(Para_0);                                Model_All.append(Model_0);    Sol_All_i.append(Sol_0); 
                    Para_0_Dry_old = Para_0;     Model_Dry_old = Model_0  ; Sol_Dry_old = Sol_0    ;   del Model_0,Sol_0,Para_0
                    #Step3: Write a big loop to finish the long experiment,    
                    Index_update_all     =[]; cycle_count =0; Index_update_all.append(cycle_count);
                    k=0; 
                    while k < SaveTimes:    # biggest loop, 
                    i=0;     cap_i = 0;
                    while i < Small_Loop:
                        # run ageing cycles, and update parameters (have another version not update)
                        Data_Pack,Paraupdate   = Cal_new_con_Update (  Sol_Dry_old,   Para_0_Dry_old )
                        [
                            Vol_EC_consumed, 
                            Vol_Elely_need, 
                            Test_V, 
                            Test_V2, 
                            Vol_Elely_add, 
                            Vol_Elely_Tot_new, 
                            Vol_Elely_JR_new, 
                            Vol_Pore_tot_new, 
                            Vol_Pore_decrease, 
                            c_e_r_new, c_EC_r_new,
                            Ratio_Dryout, Ratio_CeEC_JR, 
                            Ratio_CeLi_JR,
                            Width_new, ]= Data_Pack;
                        # Append single object to All object     
                        Vol_Elely_Tot_All.append(Vol_Elely_Tot_new*1e6);    Vol_Elely_JR_All.append(Vol_Elely_JR_new*1e6);     Vol_Pore_tot_All.append(Vol_Pore_tot_new*1e6);           
                        Ratio_CeEC_All.append(Ratio_CeEC_JR);               Ratio_CeLi_All.append(Ratio_CeLi_JR);                 Ratio_Dryout_All.append(Ratio_Dryout);
                        Vol_EC_consumed_All.append(Vol_EC_consumed*1e6);    Vol_Elely_need_All.append(Vol_Elely_need*1e6);     Width_all.append(Width_new);
                        Vol_Elely_add_All.append(Vol_Elely_add*1e6);        Vol_Pore_decrease_All.append(Vol_Pore_decrease*1e6);
                        Test_V_All.append(Test_V*1e6); Test_V2_All.append(Test_V2*1e6); 
                        c_e_r_new_All.append(c_e_r_new*1e6); c_EC_r_new_All.append(c_EC_r_new*1e6); 

                        # 3rd: run model based on new parameter and last updated solution Model  , Sol, Ratio_CeLi, Para_update, ModelExperiment, SaveAs_Cycles
                        Model_Dry_i, Sol_Dry_i   = Run_Model_Base_On_Last_Solution( 
                            Model_Dry_old  , Sol_Dry_old ,  
                            Paraupdate ,Experiment_Long, Update_Cycles )
                        Para_0_Dry_old = Paraupdate;       Model_Dry_old = Model_Dry_i;      Sol_Dry_old = Sol_Dry_i;   
                        del Paraupdate,Model_Dry_i,Sol_Dry_i
                        i += 1;   cycle_count +=  Update_Cycles; 
                        Index_update_all.append(cycle_count);

                    # run RPT, and also update parameters (otherwise will have problems)
                    Data_Pack , Paraupdate  = Cal_new_con_Update (  Sol_Dry_old,   Para_0_Dry_old   )
                    [
                            Vol_EC_consumed, 
                            Vol_Elely_need, 
                            Test_V, 
                            Test_V2, 
                            Vol_Elely_add, 
                            Vol_Elely_Tot_new, 
                            Vol_Elely_JR_new, 
                            Vol_Pore_tot_new, 
                            Vol_Pore_decrease, 
                            c_e_r_new, c_EC_r_new,
                            Ratio_Dryout, Ratio_CeEC_JR, 
                            Ratio_CeLi_JR,
                            Width_new, ]= Data_Pack;
                    # Append single object to All object     
                    Vol_Elely_Tot_All.append(Vol_Elely_Tot_new*1e6);    Vol_Elely_JR_All.append(Vol_Elely_JR_new*1e6);     Vol_Pore_tot_All.append(Vol_Pore_tot_new*1e6);           
                    Ratio_CeEC_All.append(Ratio_CeEC_JR);                  Ratio_CeLi_All.append(Ratio_CeLi_JR);                 Ratio_Dryout_All.append(Ratio_Dryout);
                    Vol_EC_consumed_All.append(Vol_EC_consumed*1e6);    Vol_Elely_need_All.append(Vol_Elely_need*1e6);     Width_all.append(Width_new);
                    Vol_Elely_add_All.append(Vol_Elely_add*1e6);        Vol_Pore_decrease_All.append(Vol_Pore_decrease*1e6); 
                    Test_V_All.append(Test_V*1e6); Test_V2_All.append(Test_V2*1e6); 
                    c_e_r_new_All.append(c_e_r_new*1e6); c_EC_r_new_All.append(c_EC_r_new*1e6); 

                    Index_update_all.append(cycle_count);

                    Model_Dry_i, Sol_Dry_i  = Run_Model_Base_On_Last_Solution_RPT( 
                        Model_Dry_old  , Sol_Dry_old ,   
                        Paraupdate,      Experiment_RPT, RPT_Cycles  )     
                    for jj in range(0,1):     # add later RPT cycle 
                        Cap_RPT_Dry_All.append    (Sol_Dry_i.cycles[-1].steps[0]["Discharge capacity [A.h]"].entries[-1] - Sol_Dry_i.cycles[-1].steps[0]["Discharge capacity [A.h]"].entries[0])
                        CapLoss_LiP_Dry_All.append(Sol_Dry_i.cycles[-1].steps[0]["Loss of capacity to negative electrode lithium plating [A.h]"].entries[-1])
                        CapLoss_SEI_Dry_All.append(Sol_Dry_i.cycles[-1].steps[0]["Loss of capacity to negative electrode SEI [A.h]"].entries[-1])
                        Cap_Pos_Dry_All.append    (Sol_Dry_i.cycles[-1].steps[0]["Positive electrode capacity [A.h]"].entries[-1])
                        Cap_Neg_Dry_All.append    (Sol_Dry_i.cycles[-1].steps[0]["Negative electrode capacity [A.h]"].entries[-1])      

                    Para_0_Dry_old = Paraupdate;    Model_Dry_old = Model_Dry_i  ;                 Sol_Dry_old = Sol_Dry_i    ;   del Paraupdate,Model_Dry_i,Sol_Dry_i
                    k += 1
                except:
                    data = openpyxl.load_workbook(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')         
                    table = data.get_sheet_by_name('Results')
                    nrows = table.max_row  # 获得行数
                    ncolumns = table.max_column  # 获得列数
                    Cap_RPT_Dry_Loss = Cap_RPT_Dry_All[-1] - Cap_RPT_Dry_All[0];
                    values = [
                        [index, cs_Neg_Init, Diff_SEI, R_SEI, 
                        Bulk_Sol_Con,D_Li_inSEI_i, c_Li_inte_ref_i,
                        Couple_SEI_LiP_i,k_LiP_i,"nan","nan",
                        "nan","nan", "nan","nan"],
                        ]
                    for i in range(1, len(values)+1):
                        for j in range(1, len(values[i-1])+1):
                            table.cell(nrows+i, j).value = values[i-1][j-1]     
                    data.save(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')
                    print("Fail for some reasons")
                else:
                    # Finish everything, try to write into excel with real results or 
                    data = openpyxl.load_workbook(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')         
                    table = data.get_sheet_by_name('Results')
                    nrows = table.max_row  # 获得行数
                    ncolumns = table.max_column  # 获得列数
                    Cap_RPT_Dry_Loss = Cap_RPT_Dry_All[-1] - Cap_RPT_Dry_All[0];
                    values = [
                        [index, cs_Neg_Init, Diff_SEI, R_SEI, 
                        Bulk_Sol_Con,D_Li_inSEI_i, c_Li_inte_ref_i,
                        Couple_SEI_LiP_i,k_LiP_i,Cap_RPT_Dry_Loss,CapLoss_LiP_Dry_All[-1],
                        CapLoss_SEI_Dry_All[-1],Vol_Elely_Tot_All[-1], Vol_Elely_JR_All[-1],Width_all[-1]],
                        ]
                    for i in range(1, len(values)+1):
                        for j in range(1, len(values[i-1])+1):
                            table.cell(nrows+i, j).value = values[i-1][j-1]     
                    data.save(BasicPath + Target+'Scan Chen2020_coupled 1170 cycles.xlsx')
       

